In [50]:
import os
import pandas as pd

import torch
import torch.nn as nn
import numpy as np

import warnings
warnings.filterwarnings(action='ignore') 

In [51]:
%cd /workspace/EmotionShortForm/

/workspace/EmotionShortForm


In [52]:
# wav 파일의 MFCC Feature와 상태정보를 합친 학습데이터를 불러옵니다.
train_df = pd.read_csv('preprocessing_csv/train_mfcc_data.csv')
test_df = pd.read_csv('preprocessing_csv/train_mfcc_data.csv')

In [53]:
train_df

,Unnamed: 0,SegmentId,time,Emotion,Valence,Arousal,mfcc_1,mfcc_2,mfcc_3,mfcc_4,...,mfcc_23,mfcc_24,mfcc_25,mfcc_26,mfcc_27,mfcc_28,mfcc_29,mfcc_30,mfcc_31,mfcc_32
0,11164,Sess26_script04_User052F_010,7.785990,neutral,3.0,3.5,-379.964142,109.689522,8.409287,14.947894,...,-0.390324,2.626545,0.437545,2.089825,0.171152,5.063381,3.834727,1.973997,-3.659752,-1.247434
1,13031,Sess24_script03_User047F_009,12.188990,happy,3.4,3.6,-316.165100,96.198570,-13.874292,23.989883,...,2.732685,5.995924,5.040682,8.586355,6.141405,8.720468,8.573661,10.320451,7.674820,4.114799
2,1147,Sess09_script01_User018M_030,4.147000,neutral,2.6,3.1,-354.932220,124.622856,13.342740,26.263084,...,-0.242917,0.397124,4.878927,5.264405,2.994150,2.870259,0.314687,2.809367,-0.628893,3.489642
3,6435,Sess14_script02_User028M_002,1.556999,neutral,3.6,3.1,-385.279846,118.912277,13.351930,38.183296,...,2.936629,-0.579203,-1.418086,2.701938,-0.128480,-0.650981,0.222955,1.449542,-1.601669,-0.025312
4,7053,Sess32_script01_User064F_049,5.443000,neutral,3.0,3.5,-351.163513,109.502029,4.125424,35.244442,...,3.420357,6.290679,3.681709,5.164492,3.685917,7.506263,7.469996,7.370858,6.551222,5.917849
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10764,5191,Sess03_script03_User005M_032,3.847000,neutral,3.2,3.3,-307.079376,110.748199,3.071705,26.098019,...,-4.210361,-1.421704,-2.658003,1.054526,-3.188072,0.515963,0.066960,0.752482,-1.321549,-3.410846
10765,13418,Sess22_script06_User043F_006,5.639000,angry,2.6,3.7,-239.924500,72.984070,-18.580250,-6.845965,...,6.675857,10.649000,7.589522,8.607011,2.216211,7.668085,-0.767406,2.716651,-3.422506,4.849663
10766,5390,Sess12_script01_User081F_018,3.157000,happy;neutral,3.7,3.6,-333.780945,95.989861,-14.832805,15.677377,...,-3.285286,1.564119,6.515084,10.374402,8.091890,14.184700,12.203178,19.104069,6.390728,4.591437
10767,860,Sess19_script02_User037M_002,6.590000,neutral,2.6,3.3,-339.677338,118.469772,10.663605,37.051121,...,-0.823455,1.826434,-0.563071,3.543955,-0.973314,3.040696,3.090068,4.243995,0.834076,1.139718


In [54]:
test_df

,Unnamed: 0,SegmentId,time,Emotion,Valence,Arousal,mfcc_1,mfcc_2,mfcc_3,mfcc_4,...,mfcc_23,mfcc_24,mfcc_25,mfcc_26,mfcc_27,mfcc_28,mfcc_29,mfcc_30,mfcc_31,mfcc_32
0,11164,Sess26_script04_User052F_010,7.785990,neutral,3.0,3.5,-379.964142,109.689522,8.409287,14.947894,...,-0.390324,2.626545,0.437545,2.089825,0.171152,5.063381,3.834727,1.973997,-3.659752,-1.247434
1,13031,Sess24_script03_User047F_009,12.188990,happy,3.4,3.6,-316.165100,96.198570,-13.874292,23.989883,...,2.732685,5.995924,5.040682,8.586355,6.141405,8.720468,8.573661,10.320451,7.674820,4.114799
2,1147,Sess09_script01_User018M_030,4.147000,neutral,2.6,3.1,-354.932220,124.622856,13.342740,26.263084,...,-0.242917,0.397124,4.878927,5.264405,2.994150,2.870259,0.314687,2.809367,-0.628893,3.489642
3,6435,Sess14_script02_User028M_002,1.556999,neutral,3.6,3.1,-385.279846,118.912277,13.351930,38.183296,...,2.936629,-0.579203,-1.418086,2.701938,-0.128480,-0.650981,0.222955,1.449542,-1.601669,-0.025312
4,7053,Sess32_script01_User064F_049,5.443000,neutral,3.0,3.5,-351.163513,109.502029,4.125424,35.244442,...,3.420357,6.290679,3.681709,5.164492,3.685917,7.506263,7.469996,7.370858,6.551222,5.917849
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10764,5191,Sess03_script03_User005M_032,3.847000,neutral,3.2,3.3,-307.079376,110.748199,3.071705,26.098019,...,-4.210361,-1.421704,-2.658003,1.054526,-3.188072,0.515963,0.066960,0.752482,-1.321549,-3.410846
10765,13418,Sess22_script06_User043F_006,5.639000,angry,2.6,3.7,-239.924500,72.984070,-18.580250,-6.845965,...,6.675857,10.649000,7.589522,8.607011,2.216211,7.668085,-0.767406,2.716651,-3.422506,4.849663
10766,5390,Sess12_script01_User081F_018,3.157000,happy;neutral,3.7,3.6,-333.780945,95.989861,-14.832805,15.677377,...,-3.285286,1.564119,6.515084,10.374402,8.091890,14.184700,12.203178,19.104069,6.390728,4.591437
10767,860,Sess19_script02_User037M_002,6.590000,neutral,2.6,3.3,-339.677338,118.469772,10.663605,37.051121,...,-0.823455,1.826434,-0.563071,3.543955,-0.973314,3.040696,3.090068,4.243995,0.834076,1.139718


In [55]:
# 학습데이터를 모델의 input으로 들어갈 x와 label로 사용할 y로 분할
train_x_df = train_df.drop(columns=['Unnamed: 0', 'SegmentId','time','Valence','Arousal','Emotion'])
train_y_df = train_df['Valence']
test_x_df = test_df.drop(columns=['Unnamed: 0', 'SegmentId','time','Valence','Arousal','Emotion'])
test_y_df = test_df['Valence']

In [56]:
# 정규화 X

train_x = torch.from_numpy(train_x_df.to_numpy()).float()
train_y = torch.from_numpy(train_y_df.to_numpy()).float()

test_x = torch.from_numpy(test_x_df.to_numpy()).float()
test_y = torch.from_numpy(test_y_df.to_numpy()).float()

In [40]:
# 정규화 O
from sklearn.preprocessing import MinMaxScaler, StandardScaler
scaler = MinMaxScaler()
scaler.fit(np.array(train_x_df))
# # ### 다 0으로 초기화됨
normalized_train_data = scaler.transform(np.array(train_x_df))
normalized_test_data = torch.tensor( scaler.transform(np.array(test_x_df))).float()


norm_train_x = torch.from_numpy(normalized_train_data).float()
train_y = torch.from_numpy(train_y_df.to_numpy()).float()

norm_test_x = torch.from_numpy(normalized_train_data).float()
test_y = torch.from_numpy(test_y_df.to_numpy()).float()
norm_train_x

tensor([[0.7807, 0.6002, 0.5256,  ..., 0.3406, 0.2676, 0.2650],
        [0.8470, 0.5264, 0.3587,  ..., 0.5966, 0.6359, 0.4704],
        [0.8067, 0.6820, 0.5625,  ..., 0.3662, 0.3661, 0.4465],
        ...,
        [0.8287, 0.5253, 0.3515,  ..., 0.8661, 0.5941, 0.4887],
        [0.8225, 0.6483, 0.5424,  ..., 0.4102, 0.4136, 0.3564],
        [0.8139, 0.6540, 0.4005,  ..., 0.4658, 0.6913, 0.4640]])

In [57]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
#         out, _ = self.lstm(x, (h0, c0))
#         out = self.fc(out[:, -1, :])
        out, (hidden_state, cell_state) = self.lstm(x, (h0, c0))
        output = self.fc(out[:,-1,:])
        hidden = self.fc(hidden_state[-1])
        return output, hidden,  out[:,-1,:]

# Grid search

In [58]:
# define the function to calculate CCC
def compute_ccc(y_true, y_pred):
    mu_true = np.mean(y_true)
    mu_pred = np.mean(y_pred)
    var_true = np.var(y_true)
    var_pred = np.var(y_pred)
    std_true = np.std(y_true)
    std_pred = np.std(y_pred)

    rho = np.corrcoef(y_true, y_pred)[0,1]

    num = 2 * rho * std_true * std_pred
    denom = var_true + var_pred + (mu_true - mu_pred)**2

    ccc = num / denom
    return ccc

In [15]:
'''
from sklearn.model_selection import KFold
# from sklearn.metrics import average_precision_score

hidden_sizes = [32, 64, 128]
num_layerss = [2, 3, 4]
num_epochss = [100, 150, 200, 250, 300]

num_folds = 3
kf = KFold(n_splits=num_folds)

results = []
for hidden_size in hidden_sizes:
    for num_layers in num_layerss:
        for num_epochs in num_epochss:
            # initialize model and optimizer
            model = LSTM(input_size, hidden_size, num_layers, output_size)
            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
            # train and evaluate using cross-validation
            fold_results = []
            for train_idx, val_idx in kf.split(train_x):
                train_x_fold, train_y_fold = train_x[train_idx], train_y[train_idx]
                
                for epoch in range(num_epochs):
                    model.train()
                    outputs = model(train_x_fold.unsqueeze(1))
                    loss = criterion(outputs, train_y_fold)
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                model.eval()
                with torch.no_grad():
                    val_outputs = model(val_x_fold.unsqueeze(1))
                    val_loss = criterion(val_outputs, val_y_fold)
                    val_predictions = val_outputs.detach().numpy().flatten()
                    val_targets = val_y_fold.detach().numpy().flatten()
                    ccc = compute_ccc(val_targets, val_predictions)
                    fold_results.append((val_loss.item(), ccc))
            # compute average performance over folds
            avg_val_loss = np.mean([x[0] for x in fold_results])
            avg_ccc = np.mean([x[1] for x in fold_results])
            results.append((hidden_size, num_layers, num_epochs, avg_val_loss, avg_ccc))
            # print results for current hyperparameter combination
            print(f"hidden_size={hidden_size}, num_layers={num_layers}, num_epochs={num_epochs}: "
                  f"val_loss={avg_val_loss:.4f}, ccc={avg_ccc:.4f}")
'''

'\nfrom sklearn.model_selection import KFold\n# from sklearn.metrics import average_precision_score\n\nhidden_sizes = [32, 64, 128]\nnum_layerss = [2, 3, 4]\nnum_epochss = [100, 150, 200, 250, 300]\n\nnum_folds = 3\nkf = KFold(n_splits=num_folds)\n\nresults = []\nfor hidden_size in hidden_sizes:\n    for num_layers in num_layerss:\n        for num_epochs in num_epochss:\n            # initialize model and optimizer\n            model = LSTM(input_size, hidden_size, num_layers, output_size)\n            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)\n            # train and evaluate using cross-validation\n            fold_results = []\n            for train_idx, val_idx in kf.split(train_x):\n                train_x_fold, train_y_fold = train_x[train_idx], train_y[train_idx]\n                \n                for epoch in range(num_epochs):\n                    model.train()\n                    outputs = model(train_x_fold.unsqueeze(1))\n                    loss = 

In [16]:
# find hyperparameters with best performance
'''
best_results = sorted(results, key=lambda x: x[4], reverse=True)
best_hidden_size, best_num_layers, best_num_epochs, _, best_ccc= best_results[0]
print(f"Best hyperparameters: hidden_size={best_hidden_size}, num_layers={best_num_layers}, "
      f"num_epochs={best_num_epochs}, ccc={best_ccc:.4f}")
'''

'\nbest_results = sorted(results, key=lambda x: x[4], reverse=True)\nbest_hidden_size, best_num_layers, best_num_epochs, _, best_ccc= best_results[0]\nprint(f"Best hyperparameters: hidden_size={best_hidden_size}, num_layers={best_num_layers}, "\n      f"num_epochs={best_num_epochs}, ccc={best_ccc:.4f}")\n'

# Model train

In [66]:
input_size = 32
hidden_size = 32 # 32
num_layers = 5 # 2
output_size = 1
learning_rate = 0.001
num_epochs = 500 # 200

In [67]:
### 정규화 
from tqdm.notebook import tqdm

model = LSTM(input_size, hidden_size, num_layers, output_size)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

min_ccc = 0
for epoch in tqdm(range(num_epochs), desc='Training', unit='epoch'):
    model.train()
    outputs,_,_ = model(norm_train_x.unsqueeze(1))
    loss = criterion(outputs, train_y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    tqdm.write('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
    model.eval()
    with torch.no_grad():
        test_predict, test_hidden, test_out = model(test_x.unsqueeze(1))
        ccc = compute_ccc(test_y.numpy(),test_predict.squeeze(1).numpy())
        if ccc > min_ccc :
                min_ccc = ccc
                print(f'min_ccc: {min_ccc}') 
                torch.save(model.state_dict(), f'./model/lstm_valence_model_best.pt')


Training:   0%|          | 0/500 [00:00<?, ?epoch/s]

Epoch [1/500], Loss: 8.8400
min_ccc: 9.637538618753532e-08
Epoch [2/500], Loss: 8.8161
Epoch [3/500], Loss: 8.7923
Epoch [4/500], Loss: 8.7686
Epoch [5/500], Loss: 8.7450
Epoch [6/500], Loss: 8.7215
Epoch [7/500], Loss: 8.6980
Epoch [8/500], Loss: 8.6745
Epoch [9/500], Loss: 8.6510
Epoch [10/500], Loss: 8.6274
Epoch [11/500], Loss: 8.6038
Epoch [12/500], Loss: 8.5802
Epoch [13/500], Loss: 8.5564
min_ccc: 9.994097951849036e-08
Epoch [14/500], Loss: 8.5325
min_ccc: 1.1033397714873902e-07
Epoch [15/500], Loss: 8.5084
min_ccc: 1.2332601799683918e-07
Epoch [16/500], Loss: 8.4841
min_ccc: 1.388411007883169e-07
Epoch [17/500], Loss: 8.4595
min_ccc: 1.564361354676366e-07
Epoch [18/500], Loss: 8.4347
min_ccc: 1.7614928212996155e-07
Epoch [19/500], Loss: 8.4095
min_ccc: 1.9793395913041376e-07
Epoch [20/500], Loss: 8.3839
min_ccc: 2.2157602067695005e-07
Epoch [21/500], Loss: 8.3578
min_ccc: 2.4660281203315094e-07
Epoch [22/500], Loss: 8.3312
min_ccc: 2.723031622787951e-07
Epoch [23/500], Loss: 8.

Epoch [236/500], Loss: 0.2291
Epoch [237/500], Loss: 0.2291
Epoch [238/500], Loss: 0.2291
Epoch [239/500], Loss: 0.2291
Epoch [240/500], Loss: 0.2291
Epoch [241/500], Loss: 0.2291
Epoch [242/500], Loss: 0.2291
Epoch [243/500], Loss: 0.2291
Epoch [244/500], Loss: 0.2291
Epoch [245/500], Loss: 0.2291
Epoch [246/500], Loss: 0.2291
Epoch [247/500], Loss: 0.2291
Epoch [248/500], Loss: 0.2291
Epoch [249/500], Loss: 0.2291
Epoch [250/500], Loss: 0.2291
Epoch [251/500], Loss: 0.2291
Epoch [252/500], Loss: 0.2291
Epoch [253/500], Loss: 0.2291
Epoch [254/500], Loss: 0.2291
Epoch [255/500], Loss: 0.2291
Epoch [256/500], Loss: 0.2291
Epoch [257/500], Loss: 0.2291
Epoch [258/500], Loss: 0.2291
Epoch [259/500], Loss: 0.2291
Epoch [260/500], Loss: 0.2291
Epoch [261/500], Loss: 0.2291
Epoch [262/500], Loss: 0.2291
Epoch [263/500], Loss: 0.2291
Epoch [264/500], Loss: 0.2291
Epoch [265/500], Loss: 0.2291
Epoch [266/500], Loss: 0.2291
Epoch [267/500], Loss: 0.2291
Epoch [268/500], Loss: 0.2291
Epoch [269

# Model evaluation

In [ ]:
valence_model = LSTM(input_size, hidden_size, num_layers, output_size)
valence_model.load_state_dict(torch.load('./model/lstm_valence_model_best.pt'))

In [31]:
model.eval()
with torch.no_grad():
    train_predict,_,_ = model(norm_train_x.unsqueeze(1))
    test_predict, test_hidden, test_out  = model(norm_test_x.unsqueeze(1))
    train_loss = criterion(train_predict, train_y)
    test_loss = criterion(test_predict, test_y)
    ccc = compute_ccc(test_y.numpy(),test_predict.squeeze(1).numpy())

#     val_predictions = test_predict.detach().numpy().flatten()
#     val_targets = test_y.detach().numpy().flatten()
#     ccc = compute_ccc(val_targets, val_predictions)
    print('Train Loss: {:.4f}'.format(train_loss.item()))
    print('Test Loss: {:.4f}'.format(test_loss.item()))
    print('ccc: {:.4f}'.format(ccc))
    print(f'Extracting 32 sized features: {test_out}') # 32 sized feature 뽑아냄
    print(f'shape: {test_out.shape}')


Train Loss: 0.2312
Test Loss: 0.2312
ccc: 0.0016
Extracting 32 sized features: tensor([[-0.4746, -0.5534,  0.6373,  ...,  0.6132,  0.6742, -0.4740],
        [-0.4790, -0.5580,  0.6417,  ...,  0.6175,  0.6781, -0.4790],
        [-0.4784, -0.5574,  0.6411,  ...,  0.6169,  0.6775, -0.4783],
        ...,
        [-0.4772, -0.5561,  0.6399,  ...,  0.6157,  0.6765, -0.4769],
        [-0.4782, -0.5572,  0.6409,  ...,  0.6167,  0.6774, -0.4781],
        [-0.4797, -0.5587,  0.6423,  ...,  0.6182,  0.6787, -0.4798]])
shape: torch.Size([10769, 32])


In [32]:
# import pickle

# filename = './model/lstm_valence_model.pkl'
# pickle.dump(model, open(filename,'wb'))

# Save model
import torch

torch.save(model.state_dict(), './model/lstm_valence_model.pt')